In [1]:
import fitz
def pdf_extract(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text
turk = pdf_extract('turkey/Turkey_9.pdf')
bela = pdf_extract('belarus/belarus_9.pdf')
ukr = pdf_extract('Ukraine/ukraine_7.pdf')+" "+pdf_extract('Ukraine/ukraine_8.pdf')+" "+pdf_extract('Ukraine/ukraine_9.pdf')

In [2]:
textbooks = [turk, bela, ukr]

# Data Cleaning

In [3]:
import re
from langdetect import detect
from langdetect import LangDetectException
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
def clean_text(post):
    if detect(post) != 'en':
        return ''
    # Make posts lowercase
    post = post.lower()

    # Remove punctuation
    post = re.sub(r'[^a-zA-Z\s]', ' ', post)
    
    # Remove words with repeated letters
    #post = re.sub(r'([a-zA-Z])\1+', r'\1', post)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(post)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    return ' '.join(filtered_text)

In [5]:
cleanTextbooks = []
for i in textbooks:
    cleanTextbooks.append(clean_text(i))

In [6]:
#These have spaces, we use these to select the male and female words
menWords = [" man ", " boy ", " male ", " brother ", " father ", " son ", " husband ", " king ", " prince ", " uncle ", " nephew ", " he ", " him ", " his ", " gentleman ", " sir ", " mr. ", " hero ", " lord ", " patriarch ", " men "]
womenWords = [" woman ", " girl ", " female ", " sister ", " mother ", " daughter ", " wife ", " queen ", " princess ", " aunt ", " niece ", " she ", " her ", " hers ", " lady ", " ma'am "," madam ", " mrs. ", " ms. ", " miss ", " heroine ", " dame ", " matriarch ", " women "]
#These have no spaces, so these can be used to throw away words during TFIDF
menWordsNoSpace = ["man", "boy", "male", "brother", "father", "son", "husband", "king", "prince", "uncle", "nephew", "he", "him", "his", "gentleman", "sir", "mr", "hero", "lord", "patriarch", "men"]
womenWordsNoSpace = ["woman", "girl", "female", "sister", "mother", "daughter", "wife", "queen", "princess", "aunt", "niece", "she", "her", "hers", "lady", "ma'am","madam", "mrs", "heroine", "dame", "matriarch", "women"]

In [7]:
menPara = [[] for _ in range(3)]
womenPara = [[] for _ in range(3)]
for i in range(0, 3):
    text = cleanTextbooks[i]
    paragraphs = [text[j:j+100] for j in range(0, len(text), 100)]
    for paragraph in paragraphs:
        for j in menWords:
            if j in paragraph:
                menPara[i].append(paragraph)
        for j in womenWords:
            if j in paragraph:
                womenPara[i].append(paragraph)

# Occurence

In [8]:
men_count = sum(len(sublist) for sublist in menPara)
print(men_count)
women_count = sum(len(sublist) for sublist in womenPara)
print(women_count)

429
342


# Firstness

In [9]:
femaleFirst = []
for sublist in womenPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1): 
                if words[i].lower() in womenWordsNoSpace and words[i + 1].lower() in menWordsNoSpace:
                    femaleFirst.append((words[i], words[i + 1]))
print(len(femaleFirst),femaleFirst)

5 [('mother', 'father'), ('queen', 'man'), ('queen', 'man'), ('daughter', 'mr'), ('daughter', 'mr')]


In [10]:
maleFirst = []
for sublist in menPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1):  
                if words[i].lower() in menWordsNoSpace and words[i + 1].lower() in womenWordsNoSpace:
                    maleFirst.append((words[i], words[i + 1]))
print(len(maleFirst),maleFirst)

26 [('brother', 'sister'), ('brother', 'sister'), ('brother', 'sister'), ('boy', 'girl'), ('men', 'women'), ('male', 'female'), ('male', 'female'), ('son', 'daughter'), ('son', 'daughter'), ('son', 'daughter'), ('son', 'daughter'), ('male', 'female'), ('male', 'female'), ('brother', 'sister'), ('nephew', 'niece'), ('sir', 'madam'), ('sir', 'madam'), ('mr', 'mrs'), ('nephew', 'niece'), ('nephew', 'niece'), ('nephew', 'niece'), ('sir', 'madam'), ('father', 'mother'), ('man', 'wife'), ('man', 'woman'), ('men', 'girl')]


In [11]:
sameLevel = {
    "siblings": {"brother", "sister"},
    "parent_child": {"father", "mother", "son", "daughter"},
    "spouses": {"husband", "wife"},
    "royalty": {"king", "queen", "prince", "princess"},
    "extended_family": {"uncle", "aunt", "nephew", "niece"},
    "personal_pronouns": {"he", "she", "him", "her", "his", "hers"},
    "titles": {"gentleman", "lady", "sir", "ma'am", "mr", "mrs", "madam"},
    "heroic_roles": {"hero", "heroine"},
    "social_roles": {"patriarch", "matriarch"},
    "general_plural": {"men", "women"},
}
flat = {word for group in sameLevel.values() for word in group}
def isSame(pair):
    return pair[0] in flat and pair[1] in flat
femaleFirst = [pair for pair in femaleFirst if isSame(pair)]

In [12]:
maleFirst = [pair for pair in maleFirst if isSame(pair)]
print(len(maleFirst))
print(len(femaleFirst))

18
3


# Sentiment analysis

In [13]:
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
fscores = analyzer.polarity_scores(fw)
mscores = analyzer.polarity_scores(fm)
print(fscores,mscores)

{'neg': 0.095, 'neu': 0.722, 'pos': 0.183, 'compound': 1.0} {'neg': 0.07, 'neu': 0.733, 'pos': 0.197, 'compound': 1.0}


# TFIDF

In [14]:
from langdetect import detect
from langdetect import LangDetectException
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
#stemming the words
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stem_nested_list(nested_list):
    return [
        [
            ' '.join(stemmer.stem(word) for word in sentence.split())
            for sentence in sublist
        ]
        for sublist in nested_list
    ]
menParaOriginal = menPara
womenParaOriginal = womenPara
menPara = stem_nested_list(menPara)
womenPara = stem_nested_list(womenPara)
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))

In [16]:
tobeTFIDF = [" ".join(" ".join(row) for row in menPara)," ".join(" ".join(row) for row in womenPara)]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(tobeTFIDF)
feature_names = vectorizer.get_feature_names_out()
dense_tfidf = tfidf_matrix.toarray()

In [17]:
#tfidf of female words
doc_index = 1
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores = sorted(word_score_pairs, key=lambda x: x[1], reverse=True)
top_300_words_fs = sorted_word_scores[:300]
top_300_words_f = [word for word, score in sorted_word_scores[:300]]

In [18]:
#tfidf of male words
doc_index = 0  
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs_m = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores_m = sorted(word_score_pairs_m, key=lambda x: x[1], reverse=True)
top_300_words_ms = sorted_word_scores_m[:300]
top_300_words_m = [word for word, score in sorted_word_scores_m[:300]]

In [19]:
#remove common words and gendered keywords
common = list(set(top_300_words_f) & set(top_300_words_m))
top_300_words_f = [word for word in top_300_words_f if word not in common]
top_300_words_f = [word for word in top_300_words_f if word not in womenWordsNoSpace]
female_300 = " ".join(top_300_words_f)
top_300_words_m = [word for word in top_300_words_m if word not in common]
top_300_words_m = [word for word in top_300_words_m if word not in menWordsNoSpace]
male_300 = " ".join(top_300_words_m)

In [20]:
print(female_300)
print(male_300)

dexter wright ladi phone interview wear dear wallac wilkin agent lawyer dawn fred home wonder elizabeth gibbon never munro knit letter met night student told tom watch enter mi perfum wig wild bag captain femal john project seen stay book children complet crew eight enough four hotel kill ls nice record ship televis design dobson guy law mobil om tomorrow winter agatha amanda becam begin britain court depart di drink en end fli happi high ho holiday hudson id kitchen lennon librari littl mean messag michael moment monday number open park person rd read return show sport step talk town victoria word yet at colleg convers corner divorc dr fall fell finish foster germani han horac itali joke krugerand lamb lifeboat marcel marion melani olymp pari pen pretti prison pro quarter samuel scarv sock steak strathclyd susan taylor terribl wa whiski wind within wrote abl anna answer bank beauti
worth jame charl gurney watt bert mari ben gold catherin news waiter nd ten salli took marpl play fiona 

In [21]:
from collections import Counter
import re
mwords = male_300.split()
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)

In [22]:
fwords = female_300.split()
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)

In [23]:
from collections import Counter
import re

# Example word lists (assuming male_300 and female_300 are defined as strings)
mwords = male_300.split()  # Male words
fwords = female_300.split()  # Female words

# Filtered words for male
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)
print("Top 40 Male Words:", topMwords)

# Filtered words for female
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)
print("Top 40 Female Words:", topFwords)


Top 40 Male Words: [('worth', 1), ('jame', 1), ('charl', 1), ('gurney', 1), ('watt', 1), ('bert', 1), ('mari', 1), ('ben', 1), ('gold', 1), ('catherin', 1), ('news', 1), ('waiter', 1), ('nd', 1), ('ten', 1), ('salli', 1), ('took', 1), ('marpl', 1), ('play', 1), ('fiona', 1), ('sgt', 1), ('ticket', 1), ('way', 1), ('alan', 1), ('back', 1), ('hour', 1), ('stare', 1), ('big', 1), ('ltd', 1), ('roll', 1), ('bottl', 1), ('cox', 1), ('jane', 1), ('list', 1), ('sergeant', 1), ('store', 1), ('anoth', 1), ('assist', 1), ('black', 1), ('five', 1), ('major', 1)]
Top 40 Female Words: [('dexter', 1), ('wright', 1), ('ladi', 1), ('phone', 1), ('interview', 1), ('wear', 1), ('dear', 1), ('wallac', 1), ('wilkin', 1), ('agent', 1), ('lawyer', 1), ('dawn', 1), ('fred', 1), ('home', 1), ('wonder', 1), ('elizabeth', 1), ('gibbon', 1), ('never', 1), ('munro', 1), ('knit', 1), ('letter', 1), ('met', 1), ('night', 1), ('student', 1), ('told', 1), ('tom', 1), ('watch', 1), ('enter', 1), ('mi', 1), ('perfum', 

# random forest

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)


proba = clf.predict_proba(X)[:, 1]  # Probability of being female
female_contributions = np.dot(X.T.toarray(), proba)
female_importance = female_contributions / np.sum(X.toarray(), axis=0)
female_importance_df = pd.DataFrame({
    "Word": feature_names,
    "FemaleImportance": female_importance
}).sort_values(by="FemaleImportance", ascending=False)

print("Top words most related to Female:")
print(female_importance_df.head(10))

Top words most related to Female:
           Word  FemaleImportance
3374     wilkin               1.0
720        dawn               1.0
902   elizabeth               1.0
1134       fred               1.0
2366   princess               1.0
1993      munro               1.0
1270        guy               1.0
764      dexter               1.0
1899         mi               1.0
1195     gibbon               1.0


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)

proba = clf.predict_proba(X)[:, 0]  # Probability of being male (class 0)


male_contributions = np.dot(X.T.toarray(), proba)
male_importance = male_contributions / np.sum(X.toarray(), axis=0)
male_importance_df = pd.DataFrame({
    "Word": feature_names,
    "MaleImportance": male_importance
}).sort_values(by="MaleImportance", ascending=False)

print("Top words most related to Male:")
print(male_importance_df.head(10))

Top words most related to Male:
          Word  MaleImportance
2893     store             1.0
377      break             1.0
2695       sgt             1.0
1628      koca             1.0
2864     stare             1.0
1690    lester             1.0
3096      took             1.0
491   champion             1.0
3411     worth             1.0
1176      gave             1.0


In [30]:
fm = ', '.join(map(str, [item for sublist in menParaOriginal for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenParaOriginal for item in sublist]))
ps = PorterStemmer()
words = word_tokenize(fw)
matching = [word for word in words if ps.stem(word) in {"cass", "aborigin","sungen"}]
matching

[]